# IMPACT paper experiments
### 1. Init
#### 1.1. Import libraries (necessary)

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../../")

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

from liriscat import utils
utils.set_seed(0)
from liriscat import dataset
from liriscat import selectionStrategy
from liriscat import CDM

import logging
import gc
import json
import torch
import pandas as pd
from importlib import reload
import IMPACT

#### 1.2. Set up the loggers (recommended)

In [2]:
utils.setuplogger(verbose = True, log_name="liriscat")

### 2. CDM prediction
#### 2.1. Training and testing, sequential version

In [3]:
import warnings
import numpy as np

gc.collect()
torch.cuda.empty_cache()

reload(utils)
reload(selectionStrategy)
reload(CDM)
reload(dataset)

<module 'liriscat.dataset' from '/home/arthurb/Programmation/liriscat/liriscat/dataset/__init__.py'>

In [34]:
config = utils.generate_eval_config(esc = 'error', valid_metric= 'mi_acc', pred_metrics = [], profile_metrics = ['doa'], save_params=False, n_query=4, num_epochs=0, batch_size=1024)
utils.set_seed(config["seed"])

config["dataset_name"] = "math2"
logging.info(config["dataset_name"])
config['learning_rate'] = 0.02026
config['lambda'] = 1.2e-5
config['d_in'] = 4
config['num_responses'] = 12
#pred_metrics,df_interp = test(config)

CUDA is available. Using GPU.
[INFO 41:50] math2


In [35]:
logging.info(f'#### {config["dataset_name"]} ####')
logging.info(f'#### config : {config} ####')
config['embs_path']='../embs/'+str(config["dataset_name"])
config['params_path']='../ckpt/'+str(config["dataset_name"])

pred_metrics = {m:[] for m in config['pred_metrics']}
profile_metrics = {m:[] for m in config['profile_metrics']}

gc.collect()
torch.cuda.empty_cache()

# Dataset downloading for doa and rm
warnings.filterwarnings("ignore", message="invalid value encountered in divide")
warnings.filterwarnings("ignore", category=RuntimeWarning)

## Concept map format : {question_id : [category_id1, category_id2, ...]}
concept_map = json.load(open(f'../datasets/2-preprocessed_data/{config["dataset_name"]}_concept_map.json', 'r'))
concept_map = {int(k): [int(x) for x in v] for k, v in concept_map.items()}

## Metadata map format : {"num_user_id": ..., "num_item_id": ..., "num_dimension_id": ...}
metadata = json.load(open(f'../datasets/2-preprocessed_data/{config["dataset_name"]}_metadata.json', 'r'))

[INFO 41:51] #### math2 ####
[INFO 41:51] #### config : {'seed': 0, 'dataset_name': 'math2', 'load_params': False, 'save_params': False, 'embs_path': '../embs/', 'params_path': '../ckpt/', 'early_stopping': True, 'esc': 'error', 'verbose_early_stopping': False, 'disable_tqdm': False, 'valid_metric': 'mi_acc', 'learning_rate': 0.02026, 'batch_size': 1024, 'num_epochs': 0, 'eval_freq': 1, 'patience': 30, 'device': device(type='cuda'), 'lambda': 1.2e-05, 'tensorboard': False, 'flush_freq': True, 'pred_metrics': [], 'profile_metrics': ['doa'], 'num_responses': 12, 'low_mem': False, 'n_query': 4, 'CDM': 'impact', 'd_in': 4} ####


In [36]:
i_fold = 0
## Dataframe columns : (user_id, question_id, response, category_id)
train_df = pd.read_csv(
    f'../datasets/2-preprocessed_data/{config["dataset_name"]}_train_{i_fold}.csv',
    encoding='utf-8', dtype={'student_id': int, 'item_id': int, "correct": float,
                                                             "dimension_id": int})
valid_df = pd.read_csv(
    f'../datasets/2-preprocessed_data/{config["dataset_name"]}_valid_{i_fold}.csv',
    encoding='utf-8', dtype={'student_id': int, 'item_id': int, "correct": float,
                                                             "dimension_id": int})
test_df = pd.read_csv(
    f'../datasets/2-preprocessed_data/{config["dataset_name"]}_test_{i_fold}.csv',
    encoding='utf-8', dtype={'student_id': int, 'item_id': int, "correct": float,
                                                             "dimension_id": int})

In [37]:
reload(dataset)
train_data = dataset.CATDataset(train_df, concept_map, metadata, config, config['batch_size'])
valid_data = dataset.evalDataset(valid_df, concept_map, metadata, config, batch_size=10000, pin_memory=True)
test_data = dataset.evalDataset(test_df, concept_map, metadata, config, batch_size=10000, pin_memory=False)

/home/arthurb/Programmation/liriscat/liriscat/dataset/dataset.py:150: UserWarning: The torch_array contains both query and the meta set !
  warnings.warn("The torch_array contains both query and the meta set !")
/home/arthurb/Programmation/liriscat/liriscat/dataset/dataset.py:155: UserWarning: The log_tensor contains both query and the meta set !
  warnings.warn("The log_tensor contains both query and the meta set !")
/home/arthurb/Programmation/liriscat/liriscat/dataset/dataset.py:160: UserWarning: The user_dict contains both query and the meta set !
  warnings.warn("The user_dict contains both query and the meta set !")


In [38]:
reload(utils)
reload(selectionStrategy)
reload(dataset)
reload(CDM)

S = selectionStrategy.Random(**config)
S.train(train_data, valid_data)

RandomModel()
[INFO 41:53] train on cuda
[INFO 41:53] -- START Training --


/home/arthurb/Programmation/liriscat/liriscat/selectionStrategy/random.py:37: UserWarning: get_params() Notimplemented
  warnings.warn('get_params() Notimplemented')
0it [00:00, ?it/s]

[INFO 41:53] 0
[INFO 41:53] 1
[INFO 41:54] 2
[INFO 41:54] 3
[INFO 41:56] 0
[INFO 41:56] 1
[INFO 41:56] 2
[INFO 41:56] 3
[INFO 41:58] 0
[INFO 41:58] 1
[INFO 41:58] 2
[INFO 41:58] 3
[INFO 41:59] 0.5831589102745056, 1.3771882057189941


1it [00:05,  5.74s/it]

[INFO 41:59] -- END Training --


In [13]:
test_data.split_query_meta(2)
S.evaluate_test(test_data)

[INFO 09:55] cuda:0


({0: {'rmse': 0.6543720960617065, 'mae': 0.4282028377056122},
  1: {'rmse': 0.6543720960617065, 'mae': 0.4282028377056122},
  2: {'rmse': 0.6543720960617065, 'mae': 0.4282028377056122},
  3: {'rmse': 0.6543720960617065, 'mae': 0.4282028377056122}},
 {0: {'doa': np.float64(0.519719267738173)},
  1: {'doa': np.float64(0.5202492812032642)},
  2: {'doa': np.float64(0.5198505497550436)},
  3: {'doa': np.float64(0.5204994047125866)}})

In [ ]:
print('test')